Open a new or existing Colab notebook.
Go to the "Runtime" menu at the top.
Select "Change runtime type."
Ensure that "Hardware accelerator" is set to "GPU."

Run pip install accelerate -U in a cell
In the top menu click Runtime → Restart Runtime
Do not rerun any cells with !pip install in them
Rerun all the other code cells and you should be good to go!

In [1]:
!pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00


In [2]:
!pip install accelerate -U


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 5.0 MB/s eta 0:00:00


In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [3]:
import transformers

print(transformers.__version__)

4.37.2


In [4]:
from datasets import load_dataset

san_data = load_dataset("sanskrit_classic")
san_data

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/342033 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 342033
    })
})

In [5]:
train_dataset = san_data["train"]
train_dataset[:10]

{'text': ['रागादि-रोगान् सततानुषक्तान् अ-शेष-काय-प्रसृतान् अ-शेषान् ।\n',
  'औत्सुक्य-मोहा-रति-दाञ् जघान यो ऽ-पूर्व-वैद्याय नमो ऽस्तु तस्मै ॥\n',
  'आयुः-कामयमानेन धर्मार्थ-सुख-साधनम् ।\n',
  'आयुर्-वेदोपदेशेषु विधेयः परम् आदरः ॥\n',
  'ब्रह्मा स्मृत्वायुषो वेदं प्रजापतिम् अजिग्रहत् ।\n',
  'सो ऽश्विनौ तौ सहस्राक्षं सो ऽत्रि-पुत्रादिकान् मुनीन् ॥\n',
  'ते ऽग्निवेशादिकांस् ते तु पृथक् तन्त्राणि तेनिरे ।\n',
  'तेभ्यो ऽति-विप्रकीर्णेभ्यः प्रायः सार-तरोच्चयः ॥\n',
  'क्रियते ऽष्टाङ्ग-हृदयं नाति-संक्षेप-विस्तरम् ।\n',
  'काय-बाल-ग्रहोर्ध्वाङ्ग-शल्य-दंष्ट्रा-जरा-वृषान् ॥\n']}

In [6]:
model_checkpoint = "distilroberta-base"

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [8]:
def tokenize_function(examples):
    return tokenizer(examples["text"], max_length=1104, truncation=True)

tokenized_datasets = train_dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

Map (num_proc=4):   0%|          | 0/342033 [00:00<?, ? examples/s]

In [9]:
tokenized_datasets

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 342033
})

In [10]:
block_size = 128

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/342033 [00:00<?, ? examples/s]

In [11]:
lm_datasets

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 193657
})

In [12]:
from transformers import AutoModelForMaskedLM

model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
from transformers import Trainer, TrainingArguments

model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-sanskrit-classic",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
)

In [14]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [15]:
lm_datasets_train = lm_datasets.select(range(20000))
lm_datasets_train

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 20000
})

In [16]:
lm_datasets_test = lm_datasets.select(range(20000, 24000))
lm_datasets_test

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 4000
})

In [17]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets_train,
    eval_dataset=lm_datasets_test,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


In [ ]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 2.06


In [ ]:
trainer.push_to_hub()

In [ ]:
model.save_pretrained("content/model")

In [ ]:
tokenizer.save_pretrained("content/tokenizer")

('content/tokenizer/tokenizer_config.json',
 'content/tokenizer/special_tokens_map.json',
 'content/tokenizer/vocab.json',
 'content/tokenizer/merges.txt',
 'content/tokenizer/added_tokens.json',
 'content/tokenizer/tokenizer.json')

In [ ]:
model = AutoModelForMaskedLM.from_pretrained("content/model")
tokenizer = AutoTokenizer.from_pretrained("content/tokenizer")

In [ ]:
input_text = "रागादि-रोगान् सततानुषक्तान् अ-शेष-काय-प्रसृतान् अ-शेषान् ।"

# Add a [MASK] token to the input text
masked_input = input_text.replace("रोगान्", "[MASK]")

# Tokenize the input
tokenized_input = tokenizer(masked_input, return_tensors="pt")


In [ ]:
import torch

# Move the input tensor to the same device as the model
tokenized_input = {key: value.to(model.device) for key, value in tokenized_input.items()}

# Get model outputs
with torch.no_grad():
    outputs = model(**tokenized_input)

# Get the predicted probabilities for each token
predictions = outputs.logits

# Get the predicted token indices and probabilities for all positions
predicted_token_indices = torch.argmax(predictions, dim=-1)
predicted_probabilities = torch.nn.functional.softmax(predictions, dim=-1)

# Remove padding tokens
predicted_token_indices_no_padding = [idx for idx in predicted_token_indices[0] if idx != tokenizer.pad_token_id]

# Decode the entire sequence of predicted tokens, skipping special tokens
predicted_tokens = tokenizer.decode(predicted_token_indices_no_padding, skip_special_tokens=True)

# Print each predicted word along with its probability
for token_index, probability in zip(predicted_token_indices_no_padding, predicted_probabilities[0]):
    predicted_word = tokenizer.decode(token_index)
    print(f"Predicted Word: {predicted_word}, Probability: {probability.item():.2f}")


NameError: name 'tokenized_input' is not defined

In [ ]:
'''# Tokenize the text
tokenized_data = tokenizer(train_dataset['text'], padding=True, truncation=True, return_tensors='pt')

# Print the tokenized output
print(tokenized_data)'''

{'input_ids': tensor([[ 2, 13,  1,  ...,  0,  0,  0],
        [ 2, 13,  1,  ...,  0,  0,  0],
        [ 2, 13,  1,  ...,  0,  0,  0],
        ...,
        [ 2, 13,  1,  ...,  0,  0,  0],
        [ 2, 13,  1,  ...,  0,  0,  0],
        [ 2, 13,  1,  ...,  0,  0,  0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


In [ ]:
'''import torch

def chunk_text(tokenized_data, max_chunk_length=512):
    input_ids = tokenized_data['input_ids']
    token_type_ids = tokenized_data['token_type_ids']
    attention_mask = tokenized_data['attention_mask']

    num_samples, seq_length = input_ids.shape

    # Initialize lists to store chunked data
    chunked_input_ids = []
    chunked_token_type_ids = []
    chunked_attention_mask = []

    for i in range(num_samples):
        # Get input_ids, token_type_ids, and attention_mask for the i-th sample
        sample_input_ids = input_ids[i]
        sample_token_type_ids = token_type_ids[i]
        sample_attention_mask = attention_mask[i]

        # Break the sample into chunks of max_chunk_length
        for start in range(0, seq_length, max_chunk_length):
            end = start + max_chunk_length
            chunked_input_ids.append(sample_input_ids[start:end])
            chunked_token_type_ids.append(sample_token_type_ids[start:end])
            chunked_attention_mask.append(sample_attention_mask[start:end])

    # Convert lists to tensors
    chunked_input_ids = torch.stack(chunked_input_ids, dim=0)
    chunked_token_type_ids = torch.stack(chunked_token_type_ids, dim=0)
    chunked_attention_mask = torch.stack(chunked_attention_mask, dim=0)

    # Return chunked tokenized data
    chunked_tokenized_data = {
        'input_ids': chunked_input_ids,
        'token_type_ids': chunked_token_type_ids,
        'attention_mask': chunked_attention_mask,
    }

    return chunked_tokenized_data

max_chunk_length = 1024  # Adjust as needed
chunked_data = chunk_text(tokenized_data, max_chunk_length)

# Now, `chunked_data` contains tokenized data broken into chunks
chunked_data'''

{'input_ids': tensor([[ 2, 13,  1,  ...,  0,  0,  0],
         [ 2, 13,  1,  ...,  0,  0,  0],
         [ 2, 13,  1,  ...,  0,  0,  0],
         ...,
         [ 2, 13,  1,  ...,  0,  0,  0],
         [ 2, 13,  1,  ...,  0,  0,  0],
         [ 2, 13,  1,  ...,  0,  0,  0]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])}

In [ ]:
'''from transformers import DataCollatorForLanguageModeling

# Use the end-of-sequence token as the padding token
tokenizer.pad_token = tokenizer.eos_token

# Create a data collator for language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,  # Set mlm to True for masked language modeling
    mlm_probability=0.15,  # Adjust mlm_probability as needed
)'''

In [ ]:
'''from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="my_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=chunked_data,
    data_collator=data_collator,
)

trainer.train()'''

KeyError: 1